<a href="https://colab.research.google.com/github/nickklos10/SerieA_Machine_Learning_Predictions_2025/blob/main/ML_SerieA_Ultimate_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob
import os

data_path = '/content/'
csv_files = glob.glob(os.path.join(data_path, 'scraped_data_*.csv'))

team_name_mapping = {
    'INTER': 'Inter Milan',
    'MILAN': 'AC Milan',
    'JUVE': 'Juventus FC',
    'NAPOLI': 'SSC Napoli',
    'ROMA': 'AS Roma',
    'LAZIO': 'SS Lazio',
    'FIORENTINA': 'ACF Fiorentina',
    'TORINO': 'Torino FC',
    'UDINESE': 'Udinese Calcio',
    'CAGLIARI': 'Cagliari Calcio',
    'GENOA': 'Genoa CFC',
    'MONZA': 'AC Monza',
    'ATALANTA': 'Atalanta BC',
    'BOLOGNA': 'Bologna FC 1909',
    'VENEZIA': 'Venezia FC',
    'COMO': 'Como 1907',
    'EMPOLI': 'Empoli FC',
    'PARMA': 'Parma Calcio 1913',
    'VERONA': 'Hellas Verona',
    'LECCE': 'US Lecce',
}

data_frames = []

for file in csv_files:
    basename = os.path.basename(file)
    team_abbr = basename.replace('scraped_data_', '').replace('.csv', '').upper()

    try:
        df = pd.read_csv(file, header=1, engine='python')
    except Exception as e:
        print(f"Error reading {file}: {e}")
        continue

    first_column = df.columns[0].lower()
    if first_column in ['0', 'unnamed: 0', 'index']:
        df = df.drop(columns=[df.columns[0]])
        print(f"Dropped first column: {df.columns.tolist()}")
    else:
        print("No unwanted first column to drop.")

    df['Team'] = team_abbr
    print(f"Added 'Team' column: {df['Team'].unique()[0]}")

    data_frames.append(df)

if data_frames:
    combined_df = pd.concat(data_frames, ignore_index=True)
    print("\nCombined DataFrame Columns:")
    print(combined_df.columns.tolist())
    print("\nCombined DataFrame Sample:")
    print(combined_df.head())
else:
    print("No data frames to concatenate. Please check your CSV files.")
    combined_df = pd.DataFrame()

if not combined_df.empty:

    combined_df['StartYear'] = combined_df['Stagione'].str[:4]

    print("\nUnique 'StartYear' values before conversion:")
    print(combined_df['StartYear'].unique())

    combined_df['StartYear'] = pd.to_numeric(combined_df['StartYear'], errors='coerce')

    print("\nUnique 'StartYear' values after conversion:")
    print(combined_df['StartYear'].unique())

    filtered_df = combined_df[combined_df['StartYear'] >= 2004]

    print("\nFiltered DataFrame (Seasons from 2004/2005 onwards):")
    print(filtered_df.tail())

    years_present = filtered_df['StartYear'].unique()
    print("\nYears Present After Filtering:")
    print(sorted(years_present))

    missing_values = filtered_df.isnull().sum()
    print("\nMissing Values in Each Column:")
    print(missing_values)

    filtered_df = filtered_df.dropna(subset=['StartYear'])

    numeric_columns = ['Gio', 'Pti', 'Vit', 'Par', 'Sco', 'GolF', 'GolS', 'DR']
    for col in numeric_columns:
        if col in filtered_df.columns:
            filtered_df[col] = pd.to_numeric(filtered_df[col], errors='coerce')

    filtered_df = filtered_df.dropna(subset=numeric_columns)

    duplicates = filtered_df.duplicated().sum()
    print(f"\nNumber of Duplicate Rows: {duplicates}")
    filtered_df = filtered_df.drop_duplicates()

    filtered_df['Team'] = filtered_df['Team'].map(team_name_mapping)

    unmapped_teams = filtered_df['Team'].isnull().sum()
    if unmapped_teams > 0:
        print(f"\nWarning: {unmapped_teams} team(s) were not found in the mapping and are set to NaN.")
    else:
        print("\nAll team names have been successfully mapped to their full names.")

    print("\nFinal Filtered DataFrame with Mapped Team Names:")
    print(filtered_df.tail())

    print("\nData has been successfully aggregated, filtered for seasons from 2004/2005 onwards, and team names have been mapped.")
else:
    print("Combined DataFrame is empty. No further processing is done.")


No unwanted first column to drop.
Added 'Team' column: GENOA
No unwanted first column to drop.
Added 'Team' column: FIORENTINA
No unwanted first column to drop.
Added 'Team' column: BOLOGNA
No unwanted first column to drop.
Added 'Team' column: VERONA
No unwanted first column to drop.
Added 'Team' column: INTER
No unwanted first column to drop.
Added 'Team' column: MONZA
No unwanted first column to drop.
Added 'Team' column: NAPOLI
No unwanted first column to drop.
Added 'Team' column: CAGLIARI
No unwanted first column to drop.
Added 'Team' column: TORINO
No unwanted first column to drop.
Added 'Team' column: LECCE
No unwanted first column to drop.
Added 'Team' column: EMPOLI
No unwanted first column to drop.
Added 'Team' column: ROMA
No unwanted first column to drop.
Added 'Team' column: ATALANTA
No unwanted first column to drop.
Added 'Team' column: PARMA
No unwanted first column to drop.
Added 'Team' column: UDINESE
No unwanted first column to drop.
Added 'Team' column: VENEZIA
No u

In [ ]:
aggregated_data_path = 'combined_filtered_2004_onwards_mapped.csv'

aggregated_df = pd.read_csv(aggregated_data_path)

print("Aggregated DataFrame:")
print(aggregated_df.tail())

Aggregated DataFrame:
      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR       Team  \
297  2021-2022   1   38   86   26    8    4    69    31  38   AC Milan   
298  2022-2023   4   38   70   20   10    8    64    43  21   AC Milan   
299  2023-2024   2   38   75   22    9    7    76    49  27   AC Milan   
300  2024-2025   -    6   11    3    2    1    14     7   7   AC Milan   
301  2024-2025   -    6    8    2    2    2     9    11  -2  Como 1907   

     StartYear  
297       2021  
298       2022  
299       2023  
300       2024  
301       2024  


In [ ]:
coaches_data_path = '/content/serie_a_coaches_grouped.csv'

coaches_df = pd.read_csv(coaches_data_path)

print("\nCoaches DataFrame:")
print(coaches_df.head())


Coaches DataFrame:
             Club  Year                 Coach Nationality  Matches  Seasons  \
0        AC Milan  2024         Paulo Fonseca    Portugal        6        1   
1        AC Monza  2024      Alessandro Nesta       Italy        6        1   
2  ACF Fiorentina  2024    Raffaele Palladino       Italy        6        1   
3         AS Roma  2024            Ivan Juric     Croatia        2        1   
4     Atalanta BC  2024  Gian Piero Gasperini       Italy        6        1   

   Finals  Titles  Points    PPM  
0       0       0      11  1.830  
1       0       0       3  0.500  
2       0       0       7  1.170  
3       0       0       6  1.875  
4       0       0       7  1.170  


In [ ]:
mapped_teams = aggregated_df['Team'].unique()

print("\nMapped Teams:")
print(mapped_teams)


Mapped Teams:
['Genoa CFC' 'ACF Fiorentina' 'Bologna FC 1909' 'Hellas Verona'
 'Inter Milan' 'AC Monza' 'SSC Napoli' 'Cagliari Calcio' 'Torino FC'
 'US Lecce' 'Empoli FC' 'AS Roma' 'Atalanta BC' 'Parma Calcio 1913'
 'Udinese Calcio' 'Venezia FC' 'Juventus FC' 'SS Lazio' 'AC Milan'
 'Como 1907']


In [ ]:
filtered_coaches_df = coaches_df[coaches_df['Club'].isin(mapped_teams)].copy()

print("\nFiltered Coaches DataFrame:")
print(filtered_coaches_df.head())


Filtered Coaches DataFrame:
             Club  Year                 Coach Nationality  Matches  Seasons  \
0        AC Milan  2024         Paulo Fonseca    Portugal        6        1   
1        AC Monza  2024      Alessandro Nesta       Italy        6        1   
2  ACF Fiorentina  2024    Raffaele Palladino       Italy        6        1   
3         AS Roma  2024            Ivan Juric     Croatia        2        1   
4     Atalanta BC  2024  Gian Piero Gasperini       Italy        6        1   

   Finals  Titles  Points    PPM  
0       0       0      11  1.830  
1       0       0       3  0.500  
2       0       0       7  1.170  
3       0       0       6  1.875  
4       0       0       7  1.170  


In [ ]:
coaches_teams = filtered_coaches_df['Club'].unique()

print("\nTeams in Coaches Dataset After Filtering:")
print(coaches_teams)

teams_not_in_aggregated = set(coaches_teams) - set(mapped_teams)
if not teams_not_in_aggregated:
    print("\nAll teams in the coaches dataset are present in the aggregated dataset.")
else:
    print("\nThe following teams are in coaches dataset but not in the aggregated dataset:")
    print(teams_not_in_aggregated)


Teams in Coaches Dataset After Filtering:
['AC Milan' 'AC Monza' 'ACF Fiorentina' 'AS Roma' 'Atalanta BC'
 'Bologna FC 1909' 'Cagliari Calcio' 'Como 1907' 'Genoa CFC'
 'Hellas Verona' 'Inter Milan' 'Juventus FC' 'Parma Calcio 1913'
 'SS Lazio' 'SSC Napoli' 'Torino FC' 'US Lecce' 'Udinese Calcio'
 'Venezia FC']

All teams in the coaches dataset are present in the aggregated dataset.


In [ ]:
filtered_coaches_df.rename(columns={'Club': 'Team'}, inplace=True)
aggregated_df.rename(columns={'StartYear': 'Year'}, inplace=True)

merged_df = pd.merge(
    aggregated_df,
    filtered_coaches_df,
    on=['Year', 'Team'],
    how='left'
)


print("\nMerged DataFrame:")
print(merged_df.head())


Merged DataFrame:
    Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR       Team  Year  \
0  2007-2008  10   38   48   13    9   16    44    52  -8  Genoa CFC  2007   
1  2008-2009   5   38   68   19   11    8    56    39  17  Genoa CFC  2008   
2  2009-2010   9   38   51   14    9   15    57    61  -4  Genoa CFC  2009   
3  2010-2011  10   38   51   14    9   15    45    47  -2  Genoa CFC  2010   
4  2011-2012  17   38   42   11    9   18    50    69 -19  Genoa CFC  2011   

                  Coach Nationality  Matches  Seasons  Finals  Titles  Points  \
0  Gian Piero Gasperini       Italy     38.0      1.0     0.0     0.0    48.0   
1  Gian Piero Gasperini       Italy     38.0      1.0     0.0     0.0    68.0   
2  Gian Piero Gasperini       Italy     38.0      1.0     0.0     0.0    51.0   
3     Davide Ballardini       Italy     28.0      1.0     0.0     0.0    40.0   
4      Alberto Malesani       Italy     19.0      1.0     0.0     0.0    23.0   

     PPM  
0  1.260  
1  

In [ ]:
row_condition = (merged_df['Team'] == 'Parma Calcio 1913') & (merged_df['Year'] == 2007)

new_data = {
    'Coach': 'Domenico Di Carlo',
    'Nationality': 'Italy',
    'Matches': 25,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 34,
    'PPM': 0.89
}

for column, value in new_data.items():
    merged_df.loc[row_condition, column] = value

print(merged_df[row_condition])

#######

empoli_2005 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2005)

empoli_2005_data = {
    'Coach': 'Mario Somma',
    'Nationality': 'Italy',
    'Matches': 22,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 45,
    'PPM': 1.18
}

for column, value in empoli_2005_data.items():
    merged_df.loc[empoli_2005, column] = value

print(merged_df[empoli_2005])

########

empoli_2006 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2006)

empoli_2006_data = {
    'Coach': 'Luigi Cagni',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 54,
    'PPM': 1.42
}

for column, value in empoli_2006_data.items():
    merged_df.loc[empoli_2006, column] = value

print(merged_df[empoli_2006])

########

empoli_2007 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2007)

empoli_2007_data = {
    'Coach': 'Luigi Cagni',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 36,
    'PPM': 0.95
}

for column, value in empoli_2007_data.items():
    merged_df.loc[empoli_2007, column] = value

print(merged_df[empoli_2007])

########

empoli_2014 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2014)

empoli_2014_data = {
    'Coach': 'Maurizio Sarri',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 42,
    'PPM': 1.11
}

for column, value in empoli_2014_data.items():
    merged_df.loc[empoli_2014, column] = value

print(merged_df[empoli_2014])

########

empoli_2015 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2015)

empoli_2015_data = {
    'Coach': 'Marco Giampaolo',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 46,
    'PPM': 1.21
}

for column, value in empoli_2015_data.items():
    merged_df.loc[empoli_2015, column] = value

print(merged_df[empoli_2015])

########

empoli_2016 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2016)

empoli_2016_data = {
    'Coach': 'Giovanni Martusciello',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 32,
    'PPM': 0.84
}

for column, value in empoli_2016_data.items():
    merged_df.loc[empoli_2016, column] = value

print(merged_df[empoli_2016])


########

empoli_2018 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2018)

empoli_2018_data = {
    'Coach': 'Aurelio Andreazzoli',
    'Nationality': 'Italy',
    'Matches': 33,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 38,
    'PPM': 1.00
}

for column, value in empoli_2018_data.items():
    merged_df.loc[empoli_2018, column] = value

print(merged_df[empoli_2018])

########

empoli_2018 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2018)

empoli_2018_data = {
    'Coach': 'Aurelio Andreazzoli',
    'Nationality': 'Italy',
    'Matches': 33,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 38,
    'PPM': 1.00
}

for column, value in empoli_2018_data.items():
    merged_df.loc[empoli_2018, column] = value

print(merged_df[empoli_2018])

########

empoli_2021 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2021)

empoli_2021_data = {
    'Coach': 'Aurelio Andreazzoli',
    'Nationality': 'Italy',
    'Matches': 33,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 41,
    'PPM': 1.08
}

for column, value in empoli_2021_data.items():
    merged_df.loc[empoli_2021, column] = value

print(merged_df[empoli_2021])


########

empoli_2022 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2022)

empoli_2022_data = {
    'Coach': 'Paolo Zanetti',
    'Nationality': 'Italy',
    'Matches': 33,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 43,
    'PPM': 1.13
}

for column, value in empoli_2022_data.items():
    merged_df.loc[empoli_2022, column] = value

print(merged_df[empoli_2022])


########

empoli_2023 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2023)

empoli_2023_data = {
    'Coach': 'Paolo Zanetti',
    'Nationality': 'Italy',
    'Matches': 33,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 36,
    'PPM': 0.95
}

for column, value in empoli_2023_data.items():
    merged_df.loc[empoli_2023, column] = value

print(merged_df[empoli_2023])

########

empoli_2024 = (merged_df['Team'] == 'Empoli FC') & (merged_df['Year'] == 2024)

empoli_2024_data = {
    'Coach': 'Roberto DAversa',
    'Nationality': 'Italy',
    'Matches': 6,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 10,
    'PPM': 1.67
}

for column, value in empoli_2024_data.items():
    merged_df.loc[empoli_2024, column] = value

print(merged_df[empoli_2024])


########

verona_2015 = (merged_df['Team'] == 'Hellas Verona') & (merged_df['Year'] == 2015)

verona_2015_data = {
    'Coach': 'Luigi Delneri',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 28,
    'PPM': 0.74
}

for column, value in verona_2015_data.items():
    merged_df.loc[verona_2015, column] = value

print(merged_df[verona_2015])

########

verona_2017 = (merged_df['Team'] == 'Hellas Verona') & (merged_df['Year'] == 2017)

verona_2017_data = {
    'Coach': 'Fabio Pecchia ',
    'Nationality': 'Italy',
    'Matches': 38,
    'Seasons': 1,
    'Finals': 0,
    'Titles': 0,
    'Points': 25,
    'PPM': 0.66
}

for column, value in verona_2017_data.items():
    merged_df.loc[verona_2017, column] = value

print(merged_df[verona_2017])

      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  \
205  2007-2008  19   38   34    7   13   18    42    62 -20   

                  Team  Year              Coach Nationality  Matches  Seasons  \
205  Parma Calcio 1913  2007  Domenico Di Carlo       Italy     25.0      1.0   

     Finals  Titles  Points   PPM  
205     0.0     0.0    34.0  0.89  
      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR       Team  Year  \
151  2005-2006   8   38   45   13    6   19    47    61 -14  Empoli FC  2005   

           Coach Nationality  Matches  Seasons  Finals  Titles  Points   PPM  
151  Mario Somma       Italy     22.0      1.0     0.0     0.0    45.0  1.18  
      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR       Team  Year  \
152  2006-2007   7   38   54   14   12   12    42    43  -1  Empoli FC  2006   

           Coach Nationality  Matches  Seasons  Finals  Titles  Points   PPM  
152  Luigi Cagni       Italy     38.0      1.0     0.0     0.0    54.0  1.42  


In [ ]:
season_2024 = merged_df[merged_df['Year'] == 2024]

pos_dash = season_2024['Pos'] == '-'

print("\nRows in 2024-2025 Season with '-' in 'Pos':")
print(season_2024[pos_dash])



Rows in 2024-2025 Season with '-' in 'Pos':
      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  \
16   2024-2025   -    6    5    1    2    3     4    10  -6   
37   2024-2025   -    6    7    1    4    1     7     7   0   
54   2024-2025   -    6    7    1    4    1     7     9  -2   
64   2024-2025   -    6    6    2    0    4    10    11  -1   
85   2024-2025   -    6   11    3    2    1    13     7   6   
88   2024-2025   -    6    3    0    3    3     4     8  -4   
106  2024-2025   -    6   13    4    1    1    11     4   7   
125  2024-2025   -    6    5    1    2    3     4    10  -6   
141  2024-2025   -    6   11    3    2    1    10     8   2   
150  2024-2025   -    6    5    1    2    3     3    11  -8   
161  2024-2025   -    6   10    2    4    0     5     2   3   
182  2024-2025   -    6    9    2    3    1     7     4   3   
201  2024-2025   -    6    7    2    1    3    11    12  -1   
215  2024-2025   -    6    5    1    2    3    10    12  -2   
236  2024-

In [ ]:
merged_df.loc[
    (merged_df['Year'] == 2024) &
    (merged_df['Pos'] == '-'),
    'Pos'
] = 0

print("\nUpdated 'Pos' Values for 2024-2025 Season:")
print(merged_df[
    (merged_df['Year'] == 2024) &
    (merged_df['Pos'] == 0)
])



Updated 'Pos' Values for 2024-2025 Season:
      Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  \
16   2024-2025   0    6    5    1    2    3     4    10  -6   
37   2024-2025   0    6    7    1    4    1     7     7   0   
54   2024-2025   0    6    7    1    4    1     7     9  -2   
64   2024-2025   0    6    6    2    0    4    10    11  -1   
85   2024-2025   0    6   11    3    2    1    13     7   6   
88   2024-2025   0    6    3    0    3    3     4     8  -4   
106  2024-2025   0    6   13    4    1    1    11     4   7   
125  2024-2025   0    6    5    1    2    3     4    10  -6   
141  2024-2025   0    6   11    3    2    1    10     8   2   
150  2024-2025   0    6    5    1    2    3     3    11  -8   
161  2024-2025   0    6   10    2    4    0     5     2   3   
182  2024-2025   0    6    9    2    3    1     7     4   3   
201  2024-2025   0    6    7    2    1    3    11    12  -1   
215  2024-2025   0    6    5    1    2    3    10    12  -2   
236  2024-2

In [ ]:
market_values_path = '/content/serie_a_market_values.csv'

market_df = pd.read_csv(market_values_path)

print("Market Values DataFrame:")
print(market_df.head())

Market Values DataFrame:
          Team  Squad Size  Average Age  Foreigners  Average Market Value  \
0  Inter Milan          26         29.0          19              25860000   
1     AC Milan          26         25.8          19              23120000   
2  Juventus FC          27         25.8          19              21300000   
3   SSC Napoli          25         27.9          14              17260000   
4  Atalanta BC          26         27.3          15              16390000   

   Total Market Value (€)  Year  
0               672300000  2024  
1               601010000  2024  
2               575150000  2024  
3               431400000  2024  
4               426100000  2024  


In [ ]:
team_replacements = {
    'FC Empoli': 'Empoli FC',
    'Parma FC': 'Parma Calcio 1913'
}

team_columns = ['Team']


for column in team_columns:
    if column in market_df.columns:
        market_df[column] = market_df[column].replace(team_replacements)

print("\nStandardized `market_df`:")
print(market_df.head())


Standardized `standardized_market_df`:
          Team  Squad Size  Average Age  Foreigners  Average Market Value  \
0  Inter Milan          26         29.0          19              25860000   
1     AC Milan          26         25.8          19              23120000   
2  Juventus FC          27         25.8          19              21300000   
3   SSC Napoli          25         27.9          14              17260000   
4  Atalanta BC          26         27.3          15              16390000   

   Total Market Value (€)  Year  
0               672300000  2024  
1               601010000  2024  
2               575150000  2024  
3               431400000  2024  
4               426100000  2024  


In [ ]:
mapped_team_names = list(team_name_mapping.values())

filtered_market_df = market_df[market_df['Team'].isin(mapped_team_names)].copy()

print("\nFiltered Market Values DataFrame (Only Mapped Teams):")
print(filtered_market_df)


Filtered Market Values DataFrame (Only Mapped Teams):
                Team  Squad Size  Average Age  Foreigners  \
0        Inter Milan          26         29.0          19   
1           AC Milan          26         25.8          19   
2        Juventus FC          27         25.8          19   
3         SSC Napoli          25         27.9          14   
4        Atalanta BC          26         27.3          15   
..               ...         ...          ...         ...   
408         SS Lazio          45         26.3          12   
413         US Lecce          33         23.8          13   
414      Atalanta BC          40         25.4           6   
416  Bologna FC 1909          40         26.3           6   
419  Cagliari Calcio          31         26.7           6   

     Average Market Value  Total Market Value (€)  Year  
0                25860000               672300000  2024  
1                23120000               601010000  2024  
2                21300000             

In [ ]:
print("\nMerged DataFrame Columns Before Market Values Merge:")
print(merged_df.columns.tolist())

final_merged_df = pd.merge(
    merged_df,
    filtered_market_df,
    on=['Year', 'Team'],
    how='left'
)

print("\nFinal Merged DataFrame with Market Values:")
print(final_merged_df.head())


Merged DataFrame Columns Before Market Values Merge:
['Stagione', 'Pos', 'Gio', 'Pti', 'Vit', 'Par', 'Sco', 'GolF', 'GolS', 'DR', 'Team', 'Year', 'Coach', 'Nationality', 'Matches', 'Seasons', 'Finals', 'Titles', 'Points', 'PPM']

Final Merged DataFrame with Market Values:
    Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  ... Seasons  \
0  2007-2008  10   38   48   13    9   16    44    52  -8  ...     1.0   
1  2008-2009   5   38   68   19   11    8    56    39  17  ...     1.0   
2  2009-2010   9   38   51   14    9   15    57    61  -4  ...     1.0   
3  2010-2011  10   38   51   14    9   15    45    47  -2  ...     1.0   
4  2011-2012  17   38   42   11    9   18    50    69 -19  ...     1.0   

   Finals Titles Points    PPM  Squad Size  Average Age  Foreigners  \
0     0.0    0.0   48.0  1.260          45         25.3          17   
1     0.0    0.0   68.0  1.790          47         23.6          15   
2     0.0    0.0   51.0  1.340          47         24.6          17 

In [ ]:
transfer_data_path = '/content/'

transfer_files = glob.glob(os.path.join(transfer_data_path, 'serie_a_transfers*.csv'))

print("Transfer Files Found:")
print(transfer_files)

Transfer Files Found:
['/content/serie_a_transfers2008.csv', '/content/serie_a_transfers2016.csv', '/content/serie_a_transfers2023.csv', '/content/serie_a_transfers2011.csv', '/content/serie_a_transfers2013.csv', '/content/serie_a_transfers2021.csv', '/content/serie_a_transfers2014.csv', '/content/serie_a_transfers2022.csv', '/content/serie_a_transfers2017.csv', '/content/serie_a_transfers2024.csv', '/content/serie_a_transfers2009.csv', '/content/serie_a_transfers2005.csv', '/content/serie_a_transfers2020.csv', '/content/serie_a_transfers2004.csv', '/content/serie_a_transfers2007.csv', '/content/serie_a_transfers2019.csv', '/content/serie_a_transfers2012.csv', '/content/serie_a_transfers2006.csv', '/content/serie_a_transfers2018.csv', '/content/serie_a_transfers2015.csv', '/content/serie_a_transfers2010.csv']


In [ ]:
transfer_dfs = []

for file in transfer_files:
    try:
        df = pd.read_csv(file)
        transfer_dfs.append(df)
        print(f"Loaded {file} successfully.")
    except Exception as e:
        print(f"Error loading {file}: {e}")

if transfer_dfs:
    all_transfers_df = pd.concat(transfer_dfs, ignore_index=True)
    print("\nAll Transfers DataFrame:")
    print(all_transfers_df.head())
else:
    print("No transfer data files loaded. Please check your files and paths.")
    all_transfers_df = pd.DataFrame()

Loaded /content/serie_a_transfers2008.csv successfully.
Loaded /content/serie_a_transfers2016.csv successfully.
Loaded /content/serie_a_transfers2023.csv successfully.
Loaded /content/serie_a_transfers2011.csv successfully.
Loaded /content/serie_a_transfers2013.csv successfully.
Loaded /content/serie_a_transfers2021.csv successfully.
Loaded /content/serie_a_transfers2014.csv successfully.
Loaded /content/serie_a_transfers2022.csv successfully.
Loaded /content/serie_a_transfers2017.csv successfully.
Loaded /content/serie_a_transfers2024.csv successfully.
Loaded /content/serie_a_transfers2009.csv successfully.
Loaded /content/serie_a_transfers2005.csv successfully.
Loaded /content/serie_a_transfers2020.csv successfully.
Loaded /content/serie_a_transfers2004.csv successfully.
Loaded /content/serie_a_transfers2007.csv successfully.
Loaded /content/serie_a_transfers2019.csv successfully.
Loaded /content/serie_a_transfers2012.csv successfully.
Loaded /content/serie_a_transfers2006.csv succes

In [ ]:
team_replacements = {
    'FC Empoli': 'Empoli FC',
    'Parma FC': 'Parma Calcio 1913'
}

team_columns = ['Team']

for column in team_columns:
    if column in all_transfers_df.columns:
        all_transfers_df[column] = all_transfers_df[column].replace(team_replacements)

print(all_transfers_df.head())

          Team Transfer Type              Player   Age    Nationality  \
0  Inter Milan            In    Ricardo Quaresma  24.0       Portugal   
1  Inter Milan            In      Sulley Muntari  23.0          Ghana   
2  Inter Milan            In             Mancini  27.0  Brazil, Italy   
3  Inter Milan            In        Luis Jiménez  24.0   Chile, Italy   
4  Inter Milan            In  Robert Acquafresca  20.0  Italy, Poland   

  Position Market Value        From Club       Fee  Year  
0       RW     23000000         FC Porto  24600000  2008  
1       CM     12000000    Portsmouth FC  14000000  2008  
2       LW     17500000          AS Roma  13000000  2008  
3       AM      9000000   Ternana Calcio  11000000  2008  
4       CF      6500000  Cagliari Calcio   5000000  2008  


In [ ]:
mapped_team_names = list(team_name_mapping.values())

mapped_team_names.append("FC Empoli")

print("Mapped Team Names:")
print(mapped_team_names)

Mapped Team Names:
['Inter Milan', 'AC Milan', 'Juventus FC', 'SSC Napoli', 'AS Roma', 'SS Lazio', 'ACF Fiorentina', 'Torino FC', 'Udinese Calcio', 'Cagliari Calcio', 'Genoa CFC', 'AC Monza', 'Atalanta BC', 'Bologna FC 1909', 'Venezia FC', 'Como 1907', 'Empoli FC', 'Parma Calcio 1913', 'Hellas Verona', 'US Lecce', 'Empoli FC', 'FC Empoli']


In [ ]:
filtered_transfers_df = all_transfers_df[all_transfers_df['Team'].isin(mapped_team_names)].copy()

print("\nFiltered Transfers DataFrame (Only Mapped Teams):")
print(filtered_transfers_df.head())


Filtered Transfers DataFrame (Only Mapped Teams):
          Team Transfer Type              Player   Age    Nationality  \
0  Inter Milan            In    Ricardo Quaresma  24.0       Portugal   
1  Inter Milan            In      Sulley Muntari  23.0          Ghana   
2  Inter Milan            In             Mancini  27.0  Brazil, Italy   
3  Inter Milan            In        Luis Jiménez  24.0   Chile, Italy   
4  Inter Milan            In  Robert Acquafresca  20.0  Italy, Poland   

  Position Market Value        From Club       Fee  Year  
0       RW     23000000         FC Porto  24600000  2008  
1       CM     12000000    Portsmouth FC  14000000  2008  
2       LW     17500000          AS Roma  13000000  2008  
3       AM      9000000   Ternana Calcio  11000000  2008  
4       CF      6500000  Cagliari Calcio   5000000  2008  


In [ ]:
filtered_transfers_df['Age'] = pd.to_numeric(filtered_transfers_df['Age'], errors='coerce')

filtered_transfers_df['Market Value'] = pd.to_numeric(filtered_transfers_df['Market Value'], errors='coerce')

filtered_transfers_df['Fee'] = pd.to_numeric(
    filtered_transfers_df['Fee'].replace('Other', 0),
    errors='coerce'
)

filtered_transfers_df['Year'] = filtered_transfers_df['Year'].astype(int)

print("\nTransfer Data Types After Conversion:")
print(filtered_transfers_df.dtypes)


Transfer Data Types After Conversion:
Team              object
Transfer Type     object
Player            object
Age              float64
Nationality       object
Position          object
Market Value     float64
From Club         object
Fee                int64
Year               int64
dtype: object


In [ ]:
# Define a function to compute the aggregation per team per year
def aggregate_transfers(df):
    aggregation = {
        'Players In': ('Transfer Type', lambda x: (x == 'In').sum()),
        'Players Out': ('Transfer Type', lambda x: (x == 'Out').sum()),
        'Average Age IN players': ('Age', lambda x: x[df['Transfer Type'] == 'In'].mean()),
        'Average Age OUT players': ('Age', lambda x: x[df['Transfer Type'] == 'Out'].mean()),
        'Market Value IN Players': ('Market Value', lambda x: x[df['Transfer Type'] == 'In'].sum()),
        'Market Value OUT Players': ('Market Value', lambda x: x[df['Transfer Type'] == 'Out'].sum()),
        'Fees Players IN': ('Fee', lambda x: x[df['Transfer Type'] == 'In'].sum()),
        'Fees Players OUT': ('Fee', lambda x: x[df['Transfer Type'] == 'Out'].sum()),
    }
    return df.groupby(['Team', 'Year']).agg(**aggregation)


transfer_summary_df = aggregate_transfers(filtered_transfers_df)


transfer_summary_df = transfer_summary_df.reset_index()


transfer_summary_df['Net Spent'] = transfer_summary_df['Fees Players OUT'] - transfer_summary_df['Fees Players IN']


print("\nAggregated Transfer Summary:")
print(transfer_summary_df.head())


Aggregated Transfer Summary:
       Team  Year  Players In  Players Out  Average Age IN players  \
0  AC Milan  2004           6            6               26.666667   
1  AC Milan  2005           9            8               26.333333   
2  AC Milan  2006          11           12               25.909091   
3  AC Milan  2007           6            5               22.833333   
4  AC Milan  2008          12           23               23.250000   

   Average Age OUT players  Market Value IN Players  Market Value OUT Players  \
0                27.166667                      0.0                       0.0   
1                27.750000               57000000.0                       0.0   
2                26.250000               61700000.0                       0.0   
3                28.400000               21000000.0                       0.0   
4                26.913043               87100000.0                  225000.0   

   Fees Players IN  Fees Players OUT  Net Spent  
0         11

In [ ]:
def aggregate_transfers_corrected(df):
    in_transfers = df[df['Transfer Type'] == 'In']
    out_transfers = df[df['Transfer Type'] == 'Out']

    summary_in = in_transfers.groupby(['Team', 'Year']).agg(
        Players_In=('Player', 'count'),
        Average_Age_IN=('Age', 'mean'),
        Market_Value_IN_Players=('Market Value', 'sum'),
        Fees_Players_IN=('Fee', 'sum')
    ).reset_index()

    summary_out = out_transfers.groupby(['Team', 'Year']).agg(
        Players_Out=('Player', 'count'),
        Average_Age_OUT=('Age', 'mean'),
        Market_Value_OUT_Players=('Market Value', 'sum'),
        Fees_Players_OUT=('Fee', 'sum')
    ).reset_index()

    summary = pd.merge(summary_in, summary_out, on=['Team', 'Year'], how='outer')

    summary['Players_In'] = summary['Players_In'].fillna(0).astype(int)
    summary['Players_Out'] = summary['Players_Out'].fillna(0).astype(int)
    summary['Average_Age_IN'] = summary['Average_Age_IN'].fillna(0)
    summary['Average_Age_OUT'] = summary['Average_Age_OUT'].fillna(0)
    summary['Market_Value_IN_Players'] = summary['Market_Value_IN_Players'].fillna(0)
    summary['Market_Value_OUT_Players'] = summary['Market_Value_OUT_Players'].fillna(0)
    summary['Fees_Players_IN'] = summary['Fees_Players_IN'].fillna(0)
    summary['Fees_Players_OUT'] = summary['Fees_Players_OUT'].fillna(0)

    summary['Net_Spent'] = summary['Fees_Players_OUT'] - summary['Fees_Players_IN']

    return summary

transfer_summary_df = aggregate_transfers_corrected(filtered_transfers_df)

# Rename columns to match the user's specifications
transfer_summary_df.rename(columns={
    'Players_In': 'Players In',
    'Players_Out': 'Players Out',
    'Average_Age_IN': 'Average Age IN players',
    'Average_Age_OUT': 'Average Age OUT players',
    'Market_Value_IN_Players': 'Market Value IN Players',
    'Market_Value_OUT_Players': 'Market Value OUT Players',
    'Fees_Players_IN': 'Fees Players IN',
    'Fees_Players_OUT': 'Fees Players OUT'
}, inplace=True)

# Reorder columns as per specifications
transfer_summary_df = transfer_summary_df[[
    'Year', 'Team', 'Players In', 'Players Out',
    'Average Age IN players', 'Average Age OUT players',
    'Market Value IN Players', 'Market Value OUT Players',
    'Fees Players IN', 'Fees Players OUT', 'Net_Spent'
]]

print("\nAggregated Transfer Summary (Corrected):")
print(transfer_summary_df.head())


Aggregated Transfer Summary (Corrected):
   Year      Team  Players In  Players Out  Average Age IN players  \
0  2004  AC Milan           6            6               26.666667   
1  2005  AC Milan           9            8               26.333333   
2  2006  AC Milan          11           12               25.909091   
3  2007  AC Milan           6            5               22.833333   
4  2008  AC Milan          12           23               23.250000   

   Average Age OUT players  Market Value IN Players  Market Value OUT Players  \
0                27.166667                      0.0                       0.0   
1                27.750000               57000000.0                       0.0   
2                26.250000               61700000.0                       0.0   
3                28.400000               21000000.0                       0.0   
4                26.913043               87100000.0                  225000.0   

   Fees Players IN  Fees Players OUT  Net_Spent  


In [ ]:
print("\nFinal Merged DataFrame Before Transfer Merge:")
print(final_merged_df.head())


Final Merged DataFrame Before Transfer Merge:
    Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  ... Seasons  \
0  2007-2008  10   38   48   13    9   16    44    52  -8  ...     1.0   
1  2008-2009   5   38   68   19   11    8    56    39  17  ...     1.0   
2  2009-2010   9   38   51   14    9   15    57    61  -4  ...     1.0   
3  2010-2011  10   38   51   14    9   15    45    47  -2  ...     1.0   
4  2011-2012  17   38   42   11    9   18    50    69 -19  ...     1.0   

   Finals Titles Points    PPM  Squad Size  Average Age  Foreigners  \
0     0.0    0.0   48.0  1.260          45         25.3          17   
1     0.0    0.0   68.0  1.790          47         23.6          15   
2     0.0    0.0   51.0  1.340          47         24.6          17   
3     0.0    0.0   40.0  1.265          45         25.2          22   
4     0.0    0.0   23.0  1.205          39         26.2          21   

   Average Market Value  Total Market Value (€)  
0               1410000        

In [ ]:
final_merged_df_with_transfers = pd.merge(
    final_merged_df,
    transfer_summary_df,
    on=['Year', 'Team'],
    how='left'
)

print("\nFinal Merged DataFrame After Transfer Merge:")
print(final_merged_df_with_transfers.head())

final_merged_df_with_transfers.to_csv('final_merged_data_with_transfers.csv', index=False)


Final Merged DataFrame After Transfer Merge:
    Stagione Pos  Gio  Pti  Vit  Par  Sco  GolF  GolS  DR  ...  \
0  2007-2008  10   38   48   13    9   16    44    52  -8  ...   
1  2008-2009   5   38   68   19   11    8    56    39  17  ...   
2  2009-2010   9   38   51   14    9   15    57    61  -4  ...   
3  2010-2011  10   38   51   14    9   15    45    47  -2  ...   
4  2011-2012  17   38   42   11    9   18    50    69 -19  ...   

  Total Market Value (€)  Players In Players Out Average Age IN players  \
0               63250000          16          23              24.000000   
1               91700000          26          16              22.346154   
2              135850000          32          22              22.281250   
3              168180000          26          29              22.807692   
4              104300000          24          32              24.208333   

   Average Age OUT players  Market Value IN Players  Market Value OUT Players  \
0                25.30434